In [1]:
from owlready2 import *
onto_path.append("/home/alice/")
onto = get_ontology("http://www.semanticweb.org/alice/ontologies/2023/3/onto#").load()

In [2]:
with onto:
    class Predicates(Thing):
        pass
    class Functions(Thing):
        pass
    class Actions(Thing):
        pass
    class Types(Thing):
        pass
    class Objects(Thing):
        pass

AllDisjoint([Predicates,Functions,Actions,Types,Objects])

AllDisjoint([onto.Predicates, onto.Functions, onto.Actions, onto.Types, onto.Objects])

In [3]:
list(onto.classes())

[onto.Predicates, onto.Functions, onto.Actions, onto.Types, onto.Objects]

In [13]:
#with onto:
 #sync_reasoner()

In [4]:
with onto:

    class has_effect_predicates(ObjectProperty):
        domain    = [Actions]
        range     = [Predicates]

    class is_effect_predicates(ObjectProperty):
        domain    = [Predicates]
        range     = [Actions]
        inverse_property = has_effect_predicates

    class has_effect_function(ObjectProperty):
        domain    = [Actions]
        range     = [Functions]

    class is_effect_function(ObjectProperty):
        domain    = [Functions]
        range     = [Actions]
        inverse_property = has_effect_function

    class has_object(ObjectProperty):
        domain    = [Predicates]
        range     = [Objects]

    class is_object(ObjectProperty):
        domain    = [Objects]
        range     = [Predicates]
        inverse_property = has_object

    class has_type(ObjectProperty):
        domain    = [Objects]
        range     = [Types]

    class is_type(ObjectProperty):
        domain    = [Types]
        range     = [Objects]
        inverse_property = has_type

In [5]:
with onto:
    class has_value(DataProperty, FunctionalProperty): #functional means that each one has a single value
        domain =[Functions]
        range = [float]
    class is_grounded(DataProperty, FunctionalProperty): #functional means that each one has a single value
        domain =[Predicates]
        range = [bool]
    class has_operator(DataProperty): #is not a functional, each predicate can have more than one operator
        #domain =[Predicates]
        range = [str]

check inverse
a = Actions("go")
p = Predicates("at")
#a.has_effect_predicates.append(p)
p.is_effect_predicates=[a]
a.has_effect_predicates
p.is_effect_predicates
is_effect_predicates.domain

In [6]:
domain_path = "/home/alice/PROPER_LPG/"
if os.path.isfile(domain_path+'/prova_domain.pddl'):
    output_path=domain_path+'/prova_domain.pddl'
    with open(output_path, "r") as domain_file:
        raw_domain = domain_file.readlines()


In [23]:
last_action=""
actions=[]
for p in raw_domain:
    if (":durative-action" in p) or (":action" in p):
        if (":durative-action" in p):
            last_action=p.replace(":durative-action","").replace("( ","").replace("\n","")
        elif (":action" in p):
            last_action=p.replace(":action","").replace("( ","").replace("\n","")
        actions.append(last_action)
actions.pop(0)
actions_objects={}
for a in actions:
    actions_objects[a]=Actions(a)

In [8]:
#FIND TYPES AND ADD TO THE ONTOLOGY
types=[]
t=False
for p in raw_domain:
        if (")" in p):
            t=False
        if t==True:
            types.append(p.replace("\t","").replace("\n",""))
        if (":types" in p):
            t=True
types_objects={}
for t in types:
   types_objects[t]=Types(t)

In [9]:
#PREDICATES
predicates=[]
t=False
for p in raw_domain:
        if (":action" in p):
            t=False
        if t==True:
            if "?" in p:
                new=p[p.find("(")+1:p.find("?")].replace(" ","")
                
            else:
                new=p[p.find("(")+1:p.find(")")].replace(" ","")
                
            if new!="":   
                predicates.append(new)
        if (":predicates" in p):
            t=True
predicates_objects={}
for t in predicates:
    predicates_objects[t]=Predicates(t)


In [10]:
predicates_objects

{'production_room': onto.production_room,
 'assembly_room': onto.assembly_room,
 'presented_task': onto.presented_task,
 'at': onto.at,
 'human_present': onto.human_present,
 'block_to_deliver': onto.block_to_deliver,
 'empty_robot': onto.empty_robot,
 'finished': onto.finished,
 'extro': onto.extro,
 'intro': onto.intro,
 'computed_e': onto.computed_e,
 'consc': onto.consc,
 'unsc': onto.unsc,
 'computed_c': onto.computed_c,
 'agree': onto.agree,
 'disagree': onto.disagree,
 'computed_a': onto.computed_a}

In [11]:
functions=[]
t=False
for p in raw_domain:
        if (":predicates" in p):
            t=False
        if t==True:
            if "?" in p:
                new=p[p.find("(")+1:p.find("?")]
                
            else:
                new=p[p.find("(")+1:p.find(")")]
                
            if new!="":   
               functions.append(new)
        if (":functions" in p):
            t=True
function_objects={}
for t in functions:
    function_objects[t]=Functions(t)

In [12]:
#connect all the relations between actions predicates and functions
t=False
last_action=""
for p in raw_domain:
    if (":durative-action" in p) or (":action" in p):
        t=False
        if (":durative-action" in p):
            last_action=p.replace(":durative-action","").replace("( ","").replace("\n","")
        elif (":action" in p):
            last_action=p.replace(":action","").replace("( ","").replace("\n","")
    if t==True:
        if ";" not in p:
            operation=p.replace("and","").replace("at end","").replace("("," ").replace(")"," ")
            pred=operation.replace("decrease","").replace("increase","").replace("assign","").replace("not","").split()
            #operation=operation.split()
           
            if pred==[]:
                continue
            else:
                   
               check=str(pred[0])
               for l in predicates:
                   if check==l:
                       actions_objects[last_action].has_effect_predicates.append(predicates_objects[check])
                       ops=last_action.replace(" ","")+" "+operation
                       predicates_objects[check].has_operator.append(ops)
                       break
                    
               for l in functions:
                   if check==l:
                       actions_objects[last_action].has_effect_function.append(function_objects[check])
                       ops=last_action.replace(" ","")+" "+operation
                       function_objects[check].has_operator.append(ops)
                       break

    if ":effect" in p:
        t=True

In [13]:
#prendo un'azione
#prendo tutti gli effetti
#se un effetto è predicato metto has_predicate_effect
#se è una funzione metto come has_function_effect

In [14]:
for i in Functions.instances():
    print(i)

ontology_new.no_of_box
ontology_new.interaction_level
ontology_new.scrupulousness_level
ontology_new.agreeableness_level
ontology_new.dur
ontology_new.no_blocks
ontology_new.max_no_blocks
ontology_new.extroversion_coefficient
ontology_new.desired_interaction
ontology_new.conscientious_coefficient
ontology_new.desired_scrupulousness
ontology_new.agreeableness_coefficient
ontology_new.desired_agreeableness
ontology_new.baseline


In [13]:
#initialize all the predicates to false and all the functions to 0
for p in predicates:
    predicates_objects[p].is_grounded=False
for f in functions:
    function_objects[f].has_value=0

In [14]:
if os.path.isfile(domain_path+'/prova_problem.pddl'):
    output_path=domain_path+'/prova_problem.pddl'
    with open(output_path, "r") as problem_file:
        raw_problem = problem_file.readlines()

In [15]:
objects=[]
associated_types=[]
t=False
for p in raw_problem:
        if (":init" in p):
            t=False
        if t==True:
            o_t=p.replace("\n","").replace(")","").replace("\t","").replace(" ","").split("-")
            if o_t[0] != "":
               print(o_t)
               objects.append(o_t[0])
               associated_types.append(o_t[1])
        if (":objects" in p):
            t=True


['r1', 'room']
['r2', 'room']


In [16]:
objects_objects={}
c=0
for t in objects:
    objects_objects[t]=Objects(t)
    objects_objects[t].has_type=[types_objects[associated_types[c]]]
    c+=1

In [17]:
#initialize the problem
t=False
for p in raw_problem:
        if (":goal" in p):
            t=False
        if t==True:
            prec=p.replace("("," ").replace(")"," ").replace("\n"," ").split(" ")
            while 1:
                try:
                    prec.remove("")
                except:
                    break
            if prec!=[]:
                if prec[0]=="=":
                    ff=prec[1]
                    function_objects[ff].has_value=float(prec[2])
                else:
                    pp=prec[0]
                    predicates_objects[pp].is_grounded=True
                    if len(prec)==2:
                         oj=prec[1]
                         predicates_objects[pp].has_object=[objects_objects[oj]]

                      
            
        if (":init" in p):
            t=True

In [18]:
onto.save()

In [ ]:
#p=Predicates("finished")
#destroy_entity(p)

data properties
        int
        float
        bool
        str (string)
        owlready2.normstr (normalized string, a single-line string)
        owlready2.locstr (localized string, a string with a language associated)
        datetime.date
        datetime.time
        datetime.datetime

In [20]:
function_objects["interaction_level"].has_operator#

['CHIT_CHAT                         increase  interaction_level 5  \n',
 'APPROACH_HUMAN                         increase  interaction_level 5  \n',
 'EXPRESS_ENTHUSIASM                         increase  interaction_level 5  \n',
 'GO_NOT_CROWDED_AREA                         decrease  interaction_level  5  \n',
 'TURN_ON_BACK                         decrease  interaction_level  5  \n',
 'COMPUTE_METRIC_INTRO             increase  interaction_level  * extroversion_coefficient  dur   \n',
 'COMPUTE_METRIC_EXTRO             decrease  interaction_level  * extroversion_coefficient  dur   \n']

In [19]:
function_objects["interaction_level"].has_value

17.0

In [24]:
#plan the first time and get the list of action
planner_path = "/home/alice/PROPER_LPG/"
os.chdir (planner_path)
command =  './lpg++ -o prova_domain.pddl -f prova_problem.pddl -n 1 -force_neighbour_insertion'
#run the planner
fd_process = subprocess.Popen([command], stdout=subprocess.PIPE, shell=True)
(out, err) = fd_process.communicate()
fd_exit = fd_process.returncode
if os.path.isfile(planner_path+'/plan_prova_problem.pddl_1.SOL'):
        output_path=planner_path+'/plan_prova_problem.pddl_1.SOL'
        with open(output_path, "r") as plan_file:
            raw_plan = plan_file.readlines()
        plan=[]
        for p in raw_plan:
           if p[0] !=";":
            plan.append(p)
        ret=plan.remove("\n") 
        while ret==None:  
            try:
              ret=plan.remove("\n")
            except:
              ret="alice"    
        actions_to_execute=[]
        for i in plan:
            actions_to_execute.append(i[i.find("(")+1:i.find(")")])
        print(actions_to_execute)
        
else:
        print("Plan not found")

['CHECK_EXTROVERSION', 'CHECK_CONSCIENTOUS', 'CHECK_DISAGREEABLE', 'REACHING_PRODUCTION_ROOM R1 R2', 'COMPUTE_METRIC_DISAGREE', 'COMPUTE_METRIC_CONSC', 'COMPUTE_METRIC_EXTRO', 'PRESENT_PRODUCTION_ROOM R2', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_CONSC', 'COMPUTE_METRIC_DISAGREE', 'CHIT_CHAT', 'REMIND_TO_PAY_ATTENTION', 'ASK_PICK_THE_BLOCK R2', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_DISAGREE', 'SAY_ROBOT_WORK_BETTER', 'COMPUTE_METRIC_CONSC', 'REACHING_ASSEMBLY_ROOM R2 R1', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_CONSC', 'COMPUTE_METRIC_DISAGREE', 'PRESENT_ASSEMBLY_ROOM R1', 'COMPUTE_METRIC_CONSC', 'REMIND_TO_PAY_ATTENTION', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_DISAGREE', 'EXPRESS_ENTHUSIASM', 'ASK_ASSEMBLY_BLOCK R1', 'COMPUTE_METRIC_CONSC', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_DISAGREE', 'REACHING_PRODUCTION_ROOM R1 R2', 'COMPUTE_METRIC_CONSC', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_DISAGREE', 'ASK_PICK_THE_BLOCK R2', 'COMPUTE_METRIC_CONSC', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METR

In [59]:
for a in actions_to_execute:
    #check preconditions if needed
    ac=a.split(" ")[0]
    params=a.split(" ")[1:]
    print("executing: "+ac)
    print(params)
    preds=actions_objects[ac].has_effect_predicates
    for p in preds:
        ops=p.has_operator
        key = list(filter(lambda x: predicates_objects[x] == p, predicates_objects))[0]
        for o in ops:
            o=o.replace("\n","").replace("\t","").split(" ")
            while 1:
                try:
                    o.remove("")
                except:
                    break
            if o[0]==ac:
                if o[1]=="not":
                    print(o, "not")
                elif o[1]==key:
                    print(o, "pred")
                else:
                    print("bhoooooooo")
    print("-------------------")

executing: CHECK_EXTROVERSION
[]
['CHECK_EXTROVERSION', 'extro'] extro
['CHECK_EXTROVERSION', 'extro'] pred
-------------------
executing: CHECK_CONSCIENTOUS
[]
['CHECK_CONSCIENTOUS', 'consc'] consc
['CHECK_CONSCIENTOUS', 'consc'] pred
-------------------
executing: CHECK_DISAGREEABLE
[]
['CHECK_DISAGREEABLE', 'disagree'] disagree
['CHECK_DISAGREEABLE', 'disagree'] pred
-------------------
executing: REACHING_PRODUCTION_ROOM
['R1', 'R2']
['REACHING_PRODUCTION_ROOM', 'not', 'computed_e'] computed_e
['REACHING_PRODUCTION_ROOM', 'not', 'computed_e'] not
['REACHING_PRODUCTION_ROOM', 'not', 'computed_c'] computed_c
['REACHING_PRODUCTION_ROOM', 'not', 'computed_c'] not
['REACHING_PRODUCTION_ROOM', 'not', 'computed_a'] computed_a
['REACHING_PRODUCTION_ROOM', 'not', 'computed_a'] not
['REACHING_PRODUCTION_ROOM', 'not', 'at', '?l1'] at
['REACHING_PRODUCTION_ROOM', 'not', 'at', '?l1'] not
['REACHING_PRODUCTION_ROOM', 'at', '?l2'] at
['REACHING_PRODUCTION_ROOM', 'at', '?l2'] pred
['REACHING_PRODU

In [55]:
def get_key(val,dict):
    for key, value in dict.items():
        if val == value:
            return key
 
    return "key doesn't exist"

In [56]:
get_key(p,predicates_objects)

'finished'

In [58]:
key = list(filter(lambda x: predicates_objects[x] == p, predicates_objects))[0]
key

'finished'